In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import gc

pd.set_option('max_columns', 300)

In [38]:
def make_scores(data):
    to_keep = ['made1', 'made2', 'made3', 'miss1', 'miss2', 'miss3', 'reb', 'turnover', 'assist', 'steal', 'block']
    df = data[data.EventType.isin(to_keep)].copy()
    to_drop = ['EventPlayerID', 'EventSubType', 'X', 'Y', 'Area']
    df.drop(to_drop, axis=1, inplace=True)
    
    df['points_made'] = 0
    df.loc[df.EventType == 'made1', 'points_made'] = 1
    df.loc[df.EventType == 'made2', 'points_made'] = 2
    df.loc[df.EventType == 'made3', 'points_made'] = 3
    df['tmp_gameID'] = df['DayNum'].astype(str) + '_' + df['WTeamID'].astype(str) + '_' + df['LTeamID'].astype(str)
    df['Final_difference'] = df['WFinalScore'] - df['LFinalScore']
    
    df = df.sort_values(by=['DayNum', 'WTeamID', 'ElapsedSeconds'])
    
    df['points'] = df.groupby(['tmp_gameID', 'EventTeamID']).points_made.cumsum() - df.points_made
    
    del df['WCurrentScore']
    del df['LCurrentScore']
    
    df.loc[df.WTeamID == df.EventTeamID, 'WCurrentScore'] = df.points
    df.loc[df.LTeamID == df.EventTeamID, 'LCurrentScore'] = df.points

    df['WCurrentScore'] = df.groupby('tmp_gameID')['WCurrentScore'].fillna(method='ffill').fillna(0)
    df['LCurrentScore'] = df.groupby('tmp_gameID')['LCurrentScore'].fillna(method='ffill').fillna(0)
    
    df['Current_difference'] = df['WCurrentScore'] - df['LCurrentScore']
    
    del df['points']
    del df['points_made']
    del df['tmp_gameID']
    
    return df


def quarter_score(data):
    df = data.copy()
    
    df['period'] = 1
    df.loc[df.ElapsedSeconds > 20 * 60, 'period'] = 2
    df.loc[df.ElapsedSeconds > 40 * 60, 'period'] = 3
    
    df['crunch'] = 0
    df.loc[(df.ElapsedSeconds > 37 * 60) & (df.ElapsedSeconds <= 40 * 60), 'crunch'] = 1
    
    df['minutes'] = df['ElapsedSeconds'] / 60
    df['tmp_gameID'] = df['DayNum'].astype(str) + '_' + df['WTeamID'].astype(str) + '_' + df['LTeamID'].astype(str)
    
    ot = ((df.groupby('tmp_gameID').minutes.max() - 40) / 5).reset_index()
    ot['n_OT'] = np.where(ot.minutes > 0, np.ceil(ot.minutes), 0)    
    half = df[df.period==1].groupby(['tmp_gameID'], as_index=False)[['WCurrentScore', 'LCurrentScore']].max()
    half['Halftime_difference'] = half['WCurrentScore'] - half['LCurrentScore']
    half.drop(['WCurrentScore', 'LCurrentScore'], axis=1, inplace=True)
    crunchtime = df[df.crunch==0].groupby(['tmp_gameID'], as_index=False)[['WCurrentScore', 'LCurrentScore']].max()
    crunchtime['3mins_difference'] = crunchtime['WCurrentScore'] - crunchtime['LCurrentScore']
    crunchtime.drop(['WCurrentScore', 'LCurrentScore'], axis=1, inplace=True)
    
    add_ons = pd.merge(ot[['tmp_gameID', 'n_OT']], half, on='tmp_gameID')
    add_ons = pd.merge(add_ons, crunchtime, on='tmp_gameID')
    
    df = pd.merge(df, add_ons, on='tmp_gameID')
    
    del df['tmp_gameID']
    del df['minutes']
    
    if data.shape[0] != df.shape[0]:
        raise KeyError('Some merge went wrong')
    
    return df


def lead_changes(data):
    df = data.copy()
    df['tmp_gameID'] = df['DayNum'].astype(str) + '_' + df['WTeamID'].astype(str) + '_' + df['LTeamID'].astype(str)
    
    changes = df.groupby('tmp_gameID').Current_difference.apply(lambda x: len(np.where(np.diff(np.sign(x)))[0])).reset_index()
    changes.rename(columns={'Current_difference': 'game_lc'}, inplace=True)
    changes_2 = df[df.period==2].groupby('tmp_gameID').Current_difference.apply(lambda x: len(np.where(np.diff(np.sign(x)))[0])).reset_index()
    changes_2.rename(columns={'Current_difference': 'half2_lc'}, inplace=True)
    changes_3 = df[df.crunch==1].groupby('tmp_gameID').Current_difference.apply(lambda x: len(np.where(np.diff(np.sign(x)))[0])).reset_index()
    changes_3.rename(columns={'Current_difference': 'crunchtime_lc'}, inplace=True)
    
    add_ons = pd.merge(changes, changes_2, on='tmp_gameID')
    add_ons = pd.merge(add_ons, changes_3, on='tmp_gameID', how='left')
    
    df = pd.merge(df, add_ons, on='tmp_gameID', how='left').fillna(0)
    
    del df['tmp_gameID']
    
    if data.shape[0] != df.shape[0]:
        raise KeyError('Some merge went wrong')
        
    return df


def _scoreinblock(data, text):
    
    df = data.groupby('tmp_gameID', as_index=False)[['WFinalScore', 'LFinalScore', 'WCurrentScore', 'LCurrentScore']].min()
    df[f'Wpoints_made_{text}'] = df['WFinalScore'] - df['WCurrentScore']
    df[f'Lpoints_made_{text}'] = df['LFinalScore'] - df['LCurrentScore']
    
    return df[['tmp_gameID', f'Wpoints_made_{text}', f'Lpoints_made_{text}']]


def _statcount(data, stat, text):
    
    tmp = data.copy()
    tmp['is_stat'] = np.where(tmp.EventType==stat, 1, 0)
    tmp = tmp.groupby(['tmp_gameID', 'EventTeamID'], as_index=False).is_stat.sum()
    
    return tmp.rename(columns={'is_stat': text})


def event_count(data):
    df = data.copy()
    df['tmp_gameID'] = df['DayNum'].astype(str) + '_' + df['WTeamID'].astype(str) + '_' + df['LTeamID'].astype(str)
    
    # points made in each block
    half2 = _scoreinblock(df[df.period==2], 'half2')
    crunch = _scoreinblock(df[df.crunch==1], 'crunchtime')
    
    add_ons = pd.merge(half2, crunch, on='tmp_gameID')
    add_ons = pd.merge(add_ons, df[['tmp_gameID', 'WTeamID', 'LTeamID']].drop_duplicates(), on='tmp_gameID')
    
    # stats in each block
    stats = ['made1', 'made2', 'made3', 'miss1', 'miss2', 'miss3', 'reb', 'turnover', 'assist', 'steal', 'block']
    
    period = 'game'    
    for stat in stats:
        name = f'{stat}_{period}'
        to_merge = _statcount(df, stat, name)
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'WTeamID', 
                                                   name: f'W{name}'}), on=['tmp_gameID', 'WTeamID'])
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'LTeamID', 
                                                   name: f'L{name}'}), on=['tmp_gameID', 'LTeamID'])
        gc.collect()
        
    period = 'half2'
    tmp = df[df.period==2]
    for stat in stats:
        name = f'{stat}_{period}'
        to_merge = _statcount(tmp, stat, name)
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'WTeamID', 
                                                   name: f'W{name}'}), on=['tmp_gameID', 'WTeamID'])
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'LTeamID', 
                                                   name: f'L{name}'}), on=['tmp_gameID', 'LTeamID'])
        gc.collect()
        
    period = 'crunchtime'
    tmp = df[df.crunch==1]
    for stat in stats:
        name = f'{stat}_{period}'
        to_merge = _statcount(tmp, stat, name)
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'WTeamID', 
                                                   name: f'W{name}'}), on=['tmp_gameID', 'WTeamID'])
        add_ons = pd.merge(add_ons, to_merge.rename(columns={'EventTeamID': 'LTeamID', 
                                                   name: f'L{name}'}), on=['tmp_gameID', 'LTeamID'])
        gc.collect()
    
    for period in ['game', 'half2', 'crunchtime']:
        # % of scores with assists
        add_ons[f'WAst_perc_{period}'] = (add_ons[f'Wassist_{period}'] / (add_ons[f'Wmade2_{period}'] + add_ons[f'Wmade3_{period}'])).fillna(0)
        add_ons[f'LAst_perc_{period}'] = (add_ons[f'Lassist_{period}'] / (add_ons[f'Lmade2_{period}'] + add_ons[f'Lmade3_{period}'])).fillna(0)
        # % scores
        add_ons[f'WFGM_perc_{period}'] = ((add_ons[f'Wmade2_{period}'] + add_ons[f'Wmade3_{period}'])
                                          / (add_ons[f'Wmade2_{period}'] + add_ons[f'Wmade3_{period}'] + 
                                             add_ons[f'Wmiss2_{period}'] + add_ons[f'Wmiss3_{period}'])).fillna(0)
        add_ons[f'LFGM_perc_{period}'] = ((add_ons[f'Lmade2_{period}'] + add_ons[f'Lmade3_{period}'])
                                          / ((add_ons[f'Lmade2_{period}'] + add_ons[f'Lmade3_{period}']) + 
                                             add_ons[f'Lmiss2_{period}'] + add_ons[f'Lmiss3_{period}'])).fillna(0)
        add_ons[f'WFGM3_perc_{period}'] = (add_ons[f'Wmade3_{period}'] / (add_ons[f'Wmade3_{period}'] + add_ons[f'Wmiss3_{period}'])).fillna(0)
        add_ons[f'LFGM3_perc_{period}'] = (add_ons[f'Lmade3_{period}'] / (add_ons[f'Lmade3_{period}'] + add_ons[f'Lmiss3_{period}'])).fillna(0)
        add_ons[f'WFTM_perc_{period}'] = (add_ons[f'Wmade1_{period}'] / (add_ons[f'Wmade1_{period}'] + add_ons[f'Wmiss1_{period}'])).fillna(0)
        add_ons[f'LFTM_perc_{period}'] = (add_ons[f'Lmade1_{period}'] / (add_ons[f'Lmade1_{period}'] + add_ons[f'Lmiss1_{period}'])).fillna(0)
        
    
    unique_cols = ['Season', 'DayNum', 'tmp_gameID', 'WTeamID', 'LTeamID', 
                   'WFinalScore', 'LFinalScore', 'Final_difference', 'n_OT', 
                   'Halftime_difference', '3mins_difference', 
                   'game_lc', 'half2_lc', 'crunchtime_lc']
    
    to_drop = ['WTeamID', 'LTeamID'] + [col for col in add_ons if 'miss' in col]
    
    df = pd.merge(df[unique_cols].drop_duplicates(), add_ons.drop(to_drop, axis=1), on='tmp_gameID')
    
    del df['tmp_gameID']
    
    return df

In [35]:
df_2020 = pd.read_csv('data/raw_men/MEvents2020.csv')

df_2020 = make_scores(df_2020)
df_2020 = quarter_score(df_2020)
df_2020 = lead_changes(df_2020)
df_2020 = event_count(df_2020)

df_2020.head()

,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,Final_difference,n_OT,Halftime_difference,3mins_difference,game_lc,half2_lc,crunchtime_lc,Wpoints_made_half2,Lpoints_made_half2,Wpoints_made_crunchtime,Lpoints_made_crunchtime,Wmade1_game,Lmade1_game,Wmade2_game,Lmade2_game,Wmade3_game,Lmade3_game,Wreb_game,Lreb_game,Wturnover_game,Lturnover_game,Wassist_game,Lassist_game,Wsteal_game,Lsteal_game,Wblock_game,Lblock_game,Wmade1_half2,Lmade1_half2,Wmade2_half2,Lmade2_half2,Wmade3_half2,Lmade3_half2,Wreb_half2,Lreb_half2,Wturnover_half2,Lturnover_half2,Wassist_half2,Lassist_half2,Wsteal_half2,Lsteal_half2,Wblock_half2,Lblock_half2,Wmade1_crunchtime,Lmade1_crunchtime,Wmade2_crunchtime,Lmade2_crunchtime,Wmade3_crunchtime,Lmade3_crunchtime,Wreb_crunchtime,Lreb_crunchtime,Wturnover_crunchtime,Lturnover_crunchtime,Wassist_crunchtime,Lassist_crunchtime,Wsteal_crunchtime,Lsteal_crunchtime,Wblock_crunchtime,Lblock_crunchtime,WAst_perc_game,LAst_perc_game,WFGM_perc_game,LFGM_perc_game,WFGM3_perc_game,LFGM3_perc_game,WFTM_perc_game,LFTM_perc_game,WAst_perc_half2,LAst_perc_half2,WFGM_perc_half2,LFGM_perc_half2,WFGM3_perc_half2,LFGM3_perc_half2,WFTM_perc_half2,LFTM_perc_half2,WAst_perc_crunchtime,LAst_perc_crunchtime,WFGM_perc_crunchtime,LFGM_perc_crunchtime,WFGM3_perc_crunchtime,LFGM3_perc_crunchtime,WFTM_perc_crunchtime,LFTM_perc_crunchtime
0,2020,1,1114,1283,67,66,1,0.0,-6.0,-1.0,16,3,3,34.0,27.0,4.0,2.0,10,12,24,15,3,8,36,31,13,10,17,10,7,7,3,3,5,5,13,5,1,4,19,11,6,7,8,3,4,4,1,0,0,2,2,0,0,0,4,1,1,1,1,0,0,1,0,0,0.629630,0.434783,0.519231,0.425926,0.214286,0.285714,0.666667,0.666667,0.571429,0.333333,0.538462,0.375000,0.125000,0.333333,0.833333,0.714286,0.50,0.0,0.500000,0.000000,0.0,0.0,0.000,1.0
1,2020,1,1116,1349,91,43,48,0.0,20.0,48.0,4,0,0,52.0,24.0,6.0,6.0,7,9,27,14,10,2,48,31,21,27,19,7,14,9,5,2,4,8,15,8,6,0,24,18,12,15,11,5,11,7,3,1,0,2,3,2,0,0,3,3,1,1,0,1,1,1,0,0,0.513514,0.437500,0.552239,0.285714,0.400000,0.080000,0.636364,0.600000,0.523810,0.625000,0.617647,0.258065,0.461538,0.000000,0.571429,0.727273,0.00,0.5,0.600000,0.666667,0.0,0.0,0.000,0.5
2,2020,1,1120,1204,83,74,9,0.0,4.0,12.0,24,0,0,42.0,37.0,9.0,10.0,18,15,22,22,7,5,44,28,21,18,13,8,6,10,2,2,15,7,12,9,1,4,23,15,8,10,4,4,1,5,1,0,5,4,1,3,0,0,5,3,0,1,0,0,0,0,0,0,0.448276,0.296296,0.500000,0.473684,0.318182,0.277778,0.720000,0.714286,0.307692,0.307692,0.464286,0.464286,0.111111,0.444444,0.789474,0.636364,0.00,0.0,0.333333,0.600000,0.0,0.0,0.625,0.8
3,2020,1,1124,1146,105,61,44,0.0,21.0,34.0,1,0,0,52.0,29.0,10.0,0.0,7,13,22,18,18,4,53,35,14,15,25,11,6,5,2,6,1,3,12,10,9,2,28,17,7,8,14,6,4,2,1,2,0,0,2,0,2,0,7,4,0,2,3,0,1,0,1,0,0.625000,0.500000,0.563380,0.360656,0.545455,0.173913,0.500000,0.541667,0.666667,0.500000,0.552632,0.363636,0.500000,0.153846,0.200000,0.500000,0.75,0.0,0.500000,0.000000,0.4,0.0,0.000,0.0
4,2020,1,1135,1136,73,71,2,0.0,1.0,0.0,30,21,8,39.0,38.0,6.0,4.0,22,13,18,17,5,8,44,40,15,12,9,14,7,9,2,0,13,5,10,9,2,5,20,18,5,5,4,7,3,2,0,0,3,2,1,1,0,0,4,4,0,1,0,0,1,0,0,0,0.391304,0.560000,0.433962,0.403226,0.294118,0.285714,0.709677,0.619048,0.333333,0.500000,0.480000,0.451613,0.285714,0.333333,0.764706,0.555556,0.00,0.0,0.333333,0.250000,0.0,0.0,0.750,0.5


In [39]:
all_events = []

for year in np.arange(2015, 2021):
    print(year)
    df = pd.read_csv(f'data/raw_men/MEvents{year}.csv')
    df = make_scores(df)
    df = quarter_score(df)
    df = lead_changes(df)
    df = event_count(df)
    all_events.append(df)
    gc.collect()

all_events = pd.concat(all_events, ignore_index=True)

all_events.head()

2015
2016
2017
2018
2019
2020


,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,Final_difference,n_OT,Halftime_difference,3mins_difference,game_lc,half2_lc,crunchtime_lc,Wpoints_made_half2,Lpoints_made_half2,Wpoints_made_crunchtime,Lpoints_made_crunchtime,Wmade1_game,Lmade1_game,Wmade2_game,Lmade2_game,Wmade3_game,Lmade3_game,Wreb_game,Lreb_game,Wturnover_game,Lturnover_game,Wassist_game,Lassist_game,Wsteal_game,Lsteal_game,Wblock_game,Lblock_game,Wmade1_half2,Lmade1_half2,Wmade2_half2,Lmade2_half2,Wmade3_half2,Lmade3_half2,Wreb_half2,Lreb_half2,Wturnover_half2,Lturnover_half2,Wassist_half2,Lassist_half2,Wsteal_half2,Lsteal_half2,Wblock_half2,Lblock_half2,Wmade1_crunchtime,Lmade1_crunchtime,Wmade2_crunchtime,Lmade2_crunchtime,Wmade3_crunchtime,Lmade3_crunchtime,Wreb_crunchtime,Lreb_crunchtime,Wturnover_crunchtime,Lturnover_crunchtime,Wassist_crunchtime,Lassist_crunchtime,Wsteal_crunchtime,Lsteal_crunchtime,Wblock_crunchtime,Lblock_crunchtime,WAst_perc_game,LAst_perc_game,WFGM_perc_game,LFGM_perc_game,WFGM3_perc_game,LFGM3_perc_game,WFTM_perc_game,LFTM_perc_game,WAst_perc_half2,LAst_perc_half2,WFGM_perc_half2,LFGM_perc_half2,WFGM3_perc_half2,LFGM3_perc_half2,WFTM_perc_half2,LFTM_perc_half2,WAst_perc_crunchtime,LAst_perc_crunchtime,WFGM_perc_crunchtime,LFGM_perc_crunchtime,WFGM3_perc_crunchtime,LFGM3_perc_crunchtime,WFTM_perc_crunchtime,LFTM_perc_crunchtime
0,2015,11,1103,1420,74,57,17,0.0,11.0,24.0,12.0,0.0,0.0,37.0,31.0,3.0,11.0,12,14,13,17,12,3,39,40,12,16,17,13,7,9,5,0,5,12,4,8,8,1,22,23,7,6,9,7,2,7,4,0,1,4,1,2,0,1,1,7,1,1,0,2,0,1,1,0,0.680000,0.650000,0.471698,0.416667,0.400000,0.250000,0.571429,0.500000,0.750000,0.777778,0.480000,0.391304,0.500000,0.333333,0.416667,0.521739,0.000000,0.666667,0.200000,0.75,0.000000,1.0,0.500000,0.666667
1,2015,11,1104,1406,82,54,28,0.0,12.0,30.0,4.0,0.0,0.0,43.0,24.0,5.0,7.0,17,11,22,11,7,7,36,36,8,22,17,9,14,2,9,2,8,7,13,7,3,1,19,19,4,12,9,3,8,1,6,1,2,2,0,2,1,0,5,3,1,0,1,0,0,1,0,1,0.586207,0.500000,0.460317,0.360000,0.304348,0.333333,0.894737,0.733333,0.562500,0.375000,0.484848,0.296296,0.300000,0.083333,0.888889,0.875000,1.000000,0.000000,0.166667,0.40,0.500000,0.0,1.000000,1.000000
2,2015,11,1112,1291,78,55,23,0.0,15.0,25.0,3.0,0.0,0.0,38.0,30.0,2.0,4.0,12,8,27,13,4,7,40,32,8,16,17,8,11,2,5,0,6,5,13,8,2,3,14,16,2,7,9,4,4,0,1,0,2,0,0,2,0,0,2,4,0,1,0,0,1,0,0,0,0.548387,0.400000,0.574074,0.384615,0.250000,0.333333,0.480000,0.666667,0.600000,0.363636,0.600000,0.458333,0.250000,0.375000,0.545455,0.714286,0.000000,0.000000,0.000000,0.50,0.000000,0.0,0.400000,0.000000
3,2015,11,1113,1152,86,50,36,0.0,12.0,32.0,6.0,0.0,0.0,46.0,22.0,8.0,4.0,19,13,23,14,7,3,44,36,11,17,17,7,7,5,4,3,14,7,13,6,2,1,29,27,4,11,7,2,6,3,3,2,1,4,2,0,1,0,7,7,0,1,1,0,1,0,0,0,0.566667,0.411765,0.612245,0.320755,0.437500,0.150000,0.558824,0.565217,0.466667,0.285714,0.555556,0.225806,0.333333,0.100000,0.560000,0.437500,0.333333,0.000000,0.500000,0.00,0.333333,0.0,0.166667,0.500000
4,2015,11,1119,1102,84,78,6,0.0,10.0,9.0,3.0,0.0,0.0,36.0,40.0,7.0,10.0,11,10,17,25,13,6,41,32,15,18,18,14,6,8,4,6,7,5,7,13,5,3,20,17,10,11,7,6,4,6,2,4,5,0,1,3,0,1,1,6,1,2,1,1,0,1,1,0,0.600000,0.451613,0.491803,0.500000,0.520000,0.315789,0.647059,0.666667,0.583333,0.375000,0.444444,0.500000,0.555556,0.300000,0.700000,0.625000,1.000000,0.250000,0.500000,0.50,0.000000,0.5,0.833333,0.000000


In [42]:
all_events.groupby('Season').DayNum.max()

Season
2015    152
2016    152
2017    152
2018    152
2019    152
2020    128
Name: DayNum, dtype: int64